In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from ipywidgets import interact

In [ ]:
df_matches=pd.read_csv(r"/kaggle/input/ipl-complete-dataset-20082020/IPL Matches 2008-2020.csv")
df_bbb=pd.read_csv(r"/kaggle/input/ipl-complete-dataset-20082020/IPL Ball-by-Ball 2008-2020.csv")

In [ ]:
df_matches.head()

In [ ]:
df_matches.shape

In [ ]:
df_bbb.head()

In [ ]:
df_bbb.shape

In [ ]:
df_matches.dtypes

In [ ]:
df_matches.describe(include="all")

In [ ]:
df_bbb.dtypes

In [ ]:
df_matches["year"]=pd.DatetimeIndex(df_matches["date"]).year
df_matches.head()

In [ ]:
fig,ax=plt.subplots(figsize=(10,5))
no_of_matches=df_matches.groupby("year")["id"].count().to_frame().rename(columns={"id":"matches"})
plt.bar(no_of_matches.index, no_of_matches.matches, width=0.5)
plt.xlabel("Year", size=14)
plt.ylabel("Matches", size=14)
plt.title("Number of matches played each year", size= 17, y=1.1)
ax.yaxis.grid()
ax.set_axisbelow(True)
plt.show()

In [ ]:
all_teams=list(df_matches["team1"].unique())
all_teams

In [ ]:
df_matches.replace(to_replace=["Delhi Daredevils", "Deccan Chargers", "Rising Pune Supergiants"], value=["Delhi Capitals","Sunrisers Hyderabad", "Rising Pune Supergiant"],inplace=True)
df_bbb.replace(to_replace=["Delhi Daredevils", "Deccan Chargers", "Rising Pune Supergiants"], value=["Delhi Capitals","Sunrisers Hyderabad", "Rising Pune Supergiant"], inplace= True)

In [ ]:
print("Matches played v/s matches won by each from 2008-2020")

team1=df_matches[["team1"]].rename(columns={"team1":"teams"})
team2=df_matches[["team2"]].rename(columns={"team2":"teams"})
teams_matches=pd.concat([team1,team2], axis=0)
matches_won=df_matches["winner"].value_counts().to_frame()
matches_won=matches_won.reset_index().rename(columns={"index":"team", "winner": "wins"})
matches_played=teams_matches["teams"].value_counts().to_frame().reset_index().rename(columns={"index":"team", "teams": "matches_played"})
win_ratio=pd.merge(matches_played, matches_won, on="team",how="inner")
win_ratio["win_ratio"]=win_ratio["wins"]/win_ratio["matches_played"]
win_ratio.sort_values("wins", ascending= False, inplace= True)
win_ratio.set_index("team", inplace=True)
win_ratio

In [ ]:
win_ratio.reset_index(inplace=True)
win_ratio.replace(to_replace=['Royal Challengers Bangalore',
 'Kings XI Punjab',
 'Mumbai Indians',
 'Kolkata Knight Riders',
 'Rajasthan Royals',
 'Chennai Super Kings',
 'Kochi Tuskers Kerala',
 'Pune Warriors',
 'Sunrisers Hyderabad',
 'Gujarat Lions',
 'Rising Pune Supergiant',
 'Delhi Capitals'],
                  value=['RCB', 'KXIP', 'MI', 'KKR', 'RR', 'CSK', 'KTK', 'PW', 'SRH', 'GL', 'RPS', 'DC'], inplace= True)
win_ratio.set_index("team", inplace=True)
fig,ax=plt.subplots(figsize=(15,6))
barwidth=0.4
br1=range(len(win_ratio.index))
br2=[i+0.4 for i in br1]

plt.bar(br1, win_ratio["matches_played"], width=0.4, label="matches_played")
plt.bar(br2, win_ratio["wins"], width=0.4, label="wins")
plt.xlabel("Team", size=14)
plt.ylabel("number of matches", size=14)
plt.title("Win v/s matches played by each teams", size=17, y=1.05)
plt.xticks([i+0.2 for i in br1], win_ratio.index)
ax.yaxis.grid()
ax.set_axisbelow(True)
plt.show()

In [ ]:
print("Finals result of each season")

winner=list()
runner=list()
result_margin=list()

for i in list(df_matches["year"].unique()):
    season=df_matches[df_matches["year"]==i]
    winner.extend(season.tail(1)["winner"])
    runner.extend(list(set(tuple(season["team1"].tail(1))+(tuple(season["team2"].tail(1)))).difference(set(season["winner"].tail(1)))))
    result_margin.extend(season["result_margin"].tail(1).astype("int").astype("str")+(" ")+season["result"].tail(1))
    
champions={"year": list(df_matches["year"].unique()), "winner":winner, "runner":runner, "result_margin": result_margin}
champions=pd.DataFrame(champions)
champions.index.name="season"
champions.reset_index(inplace=True)
champions["season"]=champions["season"]+1
champions.set_index("season", inplace=True)
champions

In [ ]:
print("Year wise points table")
@interact

def points_table_year(year=list(df_matches["year"].unique())):
    a=df_matches[df_matches["year"]==year]
    
    if (int(a.head(1)["year"])>2009):
        a=a.drop(a.tail(4).index)
    else:
        a=a.drop(a.tail(3).index)
    team1=a[["team1"]].rename(columns={"team1":"teams"})
    team2=a[["team2"]].rename(columns={"team2":"teams"})
    teams_matches=pd.concat([team1,team2], axis=0)
    matches_won=a["winner"].value_counts().to_frame()
    matches_won=matches_won.reset_index().rename(columns={"index":"team", "winner": "wins"})
    matches_played=teams_matches["teams"].value_counts().to_frame().reset_index().rename(columns={"index":"team", "teams": "matches_played"})
    points_table=pd.merge(matches_played, matches_won, on="team",how="inner")
    points_table.sort_values("wins", ascending= False, inplace= True)
    points_table["matches_played"]=[points_table["matches_played"].max()]*points_table["wins"].count()
    points_table.set_index("team", inplace=True)
    return points_table


In [ ]:
c=df_matches.groupby("year")["winner"].value_counts().to_frame()
a=pd.DataFrame()

for year in df_matches["year"].unique():
    b=pd.DataFrame(c.loc[year])
    a=pd.concat([a,b], axis=1)
    
a.columns=list(range(2008,2021))

a.drop(a.tail(4).index, inplace=True)

ax=a.fillna(0).transpose().plot(kind="line", figsize=(16,9))
ax.set_xlabel("Year", size=18)
ax.set_ylabel("Number of wins", size=18)
ax.set_title("Performance of teams over the years", y=1.05, size=22)
ax.yaxis.grid()
ax.set_axisbelow(True)
plt.show()

In [ ]:
z=list(df_matches.groupby("year")["winner"].unique())
round=np.empty(0)

for i in range(13):
    round=np.concatenate((round,z[i]))
round=pd.DataFrame(round)[0].value_counts().to_frame()

playoff=np.empty(0)

for i in list(df_matches["year"].unique()):
    a=df_matches[df_matches["year"]==i]
    
    if (int(a.head(1)["year"])>2009):
        a=a.tail(4)
        team1=a[["team1"]].rename(columns={"team1":"teams"})
        team2=a[["team2"]].rename(columns={"team2":"teams"})
        teams_matches=pd.concat([team1,team2], axis=0)
        playoff_teams=np.array(teams_matches.teams.unique())
        playoff=np.concatenate((playoff,playoff_teams))
    else:
        a=a.tail(3)
        team1=a[["team1"]].rename(columns={"team1":"teams"})
        team2=a[["team2"]].rename(columns={"team2":"teams"})
        teams_matches=pd.concat([team1,team2], axis=0)
        playoff_teams=np.array(teams_matches.teams.unique())
        playoff=np.concatenate((playoff,playoff_teams))
        
playoff=pd.DataFrame(playoff)[0].value_counts().to_frame()

qualify=pd.concat([round,playoff], axis=1)
qualify.columns=["seasons_played","playoff_qualified"]
qualify.fillna(0, inplace=True)
qualify.sort_values("playoff_qualified", ascending=False, inplace=True)

ax=qualify.plot.bar(figsize=(14,6))
ax.set_ylabel("Seasons count", size=14)
ax.set_xlabel("Teams", size=14)
ax.set_title("Number of seasons played v/s playoff_qualified", size=18, y=1.05)
ax.yaxis.grid()
ax.set_axisbelow(True)
plt.show()

In [ ]:
playoff=pd.DataFrame()
for i in list(df_matches["year"].unique()):
    a=df_matches[df_matches["year"]==i]
    
    if (int(a.head(1)["year"])>2009):
        a=a.tail(4)
        a.drop(a.tail(1).index, inplace=True)
        playoff=pd.concat([playoff,a])
    else:
        a=a.tail(3)
        a.drop(a.tail(1).index, inplace=True)
        playoff=pd.concat([playoff,a])

team1=playoff[["team1"]].rename(columns={"team1":"teams"})
team2=playoff[["team2"]].rename(columns={"team2":"teams"})
teams_matches=pd.concat([team1,team2], axis=0)
matches_won=playoff["winner"].value_counts().to_frame()
matches_won=matches_won.reset_index().rename(columns={"index":"team", "winner": "wins"})
matches_played=teams_matches["teams"].value_counts().to_frame().reset_index().rename(columns={"index":"team", "teams": "matches_played"})
play_off=pd.merge(matches_played, matches_won, on="team",how="inner")
play_off.sort_values("wins", ascending= False, inplace= True)
play_off.set_index("team", inplace=True)

ax=play_off.plot.bar(figsize=(12,6))
ax.set_xlabel("Team", size=14)
ax.set_ylabel("Matches count", size=14)
ax.set_title("Number of playoffs played v/s won by each team (excluding Finals)", size= 17, y=1.05)
ax.yaxis.grid()
ax.set_axisbelow(True)
plt.show()

In [ ]:
playoff=pd.DataFrame()
for i in list(df_matches["year"].unique()):
    a=df_matches[df_matches["year"]==i]
    a=a.tail(1)
    playoff=pd.concat([playoff,a])

team1=playoff[["team1"]].rename(columns={"team1":"teams"})
team2=playoff[["team2"]].rename(columns={"team2":"teams"})
teams_matches=pd.concat([team1,team2], axis=0)
matches_won=playoff["winner"].value_counts().to_frame()
matches_won=matches_won.reset_index().rename(columns={"index":"team", "winner": "final_wins"})
matches_played=teams_matches["teams"].value_counts().to_frame().reset_index().rename(columns={"index":"team", "teams": "finals_played"})
play_off=pd.merge(matches_played, matches_won, on="team",how="inner")
play_off.sort_values("final_wins", ascending= False, inplace= True)
play_off.set_index("team", inplace=True)

ax=play_off.plot.bar(figsize=(12,6))
ax.set_xlabel("Team", size=14)
ax.set_ylabel("Count", size=14)
ax.set_title("Number of finals played v/s won by each team", size= 17,y=1.05)
ax.yaxis.grid()
ax.set_axisbelow(True)
plt.show()

In [ ]:
homeground=pd.DataFrame()
for j in df_matches["winner"].dropna().unique():
    ground=pd.DataFrame(df_matches.groupby("winner")["city"].value_counts().loc[j].head(1)).rename(columns={"city":"wins"}).reset_index()
    j=pd.Series(j)
    home=pd.concat([j,ground], axis=1)
    homeground=pd.concat([homeground,home])
    
homeground.rename(columns={0:'team', 'city':'homeground'},inplace=True)

matches=pd.DataFrame()
for team in df_matches["team2"].dropna().unique():
    a=pd.DataFrame()
    for index,row in df_matches.iterrows():
        if ((row['team1']==team or row['team2']==team) and (row['city']==homeground[homeground["team"]==team]["homeground"].iloc[0])):
            team1=row[["team1"]]
            team2=row[["team2"]]
            teams_matches=pd.concat([team1,team2], axis=0)
            a=pd.concat([a,teams_matches], axis=0)
        
    x=a[0].value_counts().head(1)
    matches=pd.concat([matches,x], axis=0)
    
matches=matches.reset_index().rename(columns={"index":"team", 0:"matches_played"})
matches['matches_played']=matches['matches_played'].astype("int")

homeground_matches=pd.merge(matches, homeground, how="inner", on="team")
homeground_matches=homeground_matches[['team', 'homeground', 'matches_played', 'wins']]
homeground_matches['win_ratio']=homeground_matches['wins']/homeground_matches['matches_played']*100
homeground_matches.drop(homeground_matches.tail(4).index, inplace=True)
homeground_matches.sort_values("win_ratio", ascending=False, inplace=True)
homeground_matches.set_index("team", inplace=True)

print("Teams & Their Homeground")
for team, homeground in zip(list(homeground_matches.index), list(homeground_matches['homeground'])):
    print(team,":",homeground)
    
ax=homeground_matches[['matches_played', 'wins', 'win_ratio']].plot.bar(figsize=(16,6))
ax.set_xlabel("Team", size=14)
ax.set_ylabel("Count", size=14)
ax.set_title("Win ratio of teams on their homeground", size=17, y=1.05)
ax.yaxis.grid()
ax.set_axisbelow(True)
plt.show()

In [ ]:
toss_decision=[]
for team in df_matches['team1'].unique():
    batting_first=0
    batting_second=0
    for index,row in df_matches.iterrows():
        if row['toss_winner']==team:
            if row['toss_decision']=="bat":
                batting_first+=1
            else:
                batting_second+=1
                
    x=[team,batting_first,batting_second]
    toss_decision.append(x)
toss_decision=pd.DataFrame(toss_decision, columns=['team', 'batting_first', 'batting_second']).set_index('team')
toss_decision.drop(toss_decision.tail(4).index, inplace=True)
toss_decision['total_tosses_won']=toss_decision['batting_first']+toss_decision['batting_second']
toss_decision.sort_values('total_tosses_won', ascending=False, inplace=True)

ax=toss_decision[['batting_first', 'batting_second']].plot.bar(figsize=(15,6))
ax.set_xlabel('Team', size=14)
ax.set_ylabel("Count", size=14)
ax.set_title("Decision taken by teams after winning tosses",size=17, y=1.05)
ax.yaxis.grid()
ax.set_axisbelow(True)
plt.show()

In [ ]:
tosswin_win_ratio=[]
for team in df_matches['team1'].unique():
    a=0
    b=0
    for index,row in df_matches.iterrows():
        if row['toss_winner']==team:
            a=a+1
            if row["winner"]==row['toss_winner']:
                b=b+1
    x=[team,a,b]
    tosswin_win_ratio.append(x)

tosswin_win_ratio=pd.DataFrame(tosswin_win_ratio, columns=['team', 'tosses_won', 'matches_won']).set_index("team")
tosswin_win_ratio.drop(tosswin_win_ratio.tail(4).index, inplace=True)
tosswin_win_ratio['win_ratio']=tosswin_win_ratio['matches_won']/tosswin_win_ratio['tosses_won']*100
tosswin_win_ratio.sort_values("win_ratio", ascending=False, inplace=True)

ax=tosswin_win_ratio.plot.bar(figsize=(15,6))
ax.set_xlabel("Team", size=14)
ax.set_title("Matches won by winning tosses by each team", size=17, y=1.05)
ax.yaxis.grid()
ax.set_axisbelow(True)
plt.show()

In [ ]:
bat1=[]
for team in df_matches['team1'].unique():
    batting_first=0
    batting_second=0
    for index,row in df_matches.iterrows():
        if row['winner']==team:
            if row['toss_decision']=="bat":
                batting=row['toss_winner']
                if row['winner']==batting:
                    batting_first+=1
                else:
                    batting_second+=1
            else:
                batting=list({row['team1'],row['team2']}.difference({row['toss_winner']}))[0]
                if row['winner']==batting:
                    batting_first+=1
                else:
                    batting_second+=1

    x=[team,batting_first, batting_second]
    bat1.append(x)
bat1=pd.DataFrame(bat1, columns=['team', 'batting_first_wins', 'batting_second_wins']).set_index("team")
bat1['total_matches']=win_ratio['matches_played']
bat1.drop(bat1.tail(4).index, inplace=True)

bat2=[]
for team in df_matches['team1'].unique():
    batting_first=0
    batting_second=0
    for index,row in df_matches.iterrows():
        if row['team1']==team or row['team2']==team:
            if row['toss_winner']==team:
                if row['toss_decision']=='bat':
                    batting_first+=1
                else:
                    batting_second+=1
            else:
                if row['toss_decision']=='bat':
                    batting_second+=1
                else:
                    batting_first+=1
                    
    x=[team,batting_first, batting_second]
    bat2.append(x)
bat2=pd.DataFrame(bat2, columns=['team', 'batting_first', 'batting_second']).set_index('team')
bat2.drop(bat2.tail(4).index, inplace=True)

bat=pd.concat([bat1,bat2], axis=1)
bat['total_wins']=bat['batting_first_wins']+bat['batting_second_wins']
bat['batting_first_win_ratio']=bat['batting_first_wins']/bat['batting_first']*100
bat['batting_second_win_ratio']=bat['batting_second_wins']/bat['batting_second']*100
bat.sort_values('total_wins', ascending=False, inplace=True)

ax=bat[['batting_first_wins', 'batting_second_wins', 'batting_first_win_ratio', 'batting_second_win_ratio']].plot.bar(figsize=(15,6)) 
ax.set_xlabel("Team", size=14)
ax.set_title("Batting first & batting second win ratio for each team", size=17, y=1.05)
ax.yaxis.grid()
ax.set_axisbelow(True)
ax.set_yticks(range(0,75,10))
plt.show()     